# Modeling
steps:
- Preprocessing the test set like the train data
- Scaling the data (test and train)
- function for model_evaluation
- function for model building
- 

In [14]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict, cross_validate
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import classification_report, confusion_matrix, roc_curve
from sklearn.metrics import fbeta_score, make_scorer

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

RSEED = 42

## Scaling the data

## Modeling

In [21]:
df = pd.read_csv('data/Train.csv')

In [22]:
X_train = df.drop('target', axis=1)
y_train = df.target

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train)

In [26]:
# Beispiel
# testing of different models

# root mean squared error
# r^2


def model_evaluation(clf, X_train, y_train, scoring=(r2_score, mean_squared_error)):

    results = {}
    scores = cross_validate(clf, X_train, y_train, scoring=scoring, cv=5, n_jobs=-1)
    results = {key: [value.mean().round(4), value.std().round(4)] for key, value in scores.items()}

    #del results['fit_time']
    #del results['score_time']

    return results

In [27]:
# Beispiel
list_of_clf = [LinearRegression(), KNeighborsClassifier(), RandomForestClassifier(random_state=RSEED)]

for clf in list_of_clf:
    results = model_evaluation(clf, (r2_score, mean_squared_error), X_train, y_train)
    print(clf)
    print('R^2 (mean, std): ', results["test_r2"])
    print('RMSE (mean, std): ', -results["test_neg_mean_squared_error"])
    print('----'*10)

ValueError: Found input variables with inconsistent numbers of samples: [2, 22917]

## Hyperparameter Tuning

- LinearRegression - GradientDescent
- KNN - GridSearchCV?
- Decision Tree - GridSearchCV?

## Evaluation

of different models

## Error Analysis